[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/cli/elasticity.ipynb)

# Elasticity

## Set up environment (optional)

These steps are required to run this tutorial with Google Colab. To do so, uncomment and run the cell below.

This will replace pre-installed versions of `numpy` and `torch` in Colab with versions that are known to be compatible with `janus-core`.

It may be possible to skip the steps that uninstall and reinstall `torch`, which will save a considerable amount of time.

These instructions but may work for other systems too, but it is typically preferable to prepare a virtual environment separately before running this notebook if possible.

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install janus-core[mace,visualise] data-tutorials --system # Install janus-core with MACE and WeasWidget, and data-tutorials

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

To ensure you have the latest version of `janus-core` installed, compare the output of the following cell to the latest version available at https://pypi.org/project/janus-core/

In [ ]:
from janus_core import __version__

print(__version__)

## Prepare modules

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from ase.build import nanotube, bulk
from ase.lattice.cubic import Diamond

from ase.io import write, read
from weas_widget import WeasWidget

Path("../data").mkdir(exist_ok=True)

### Generation of samples

In ```janus_core``` we can calculate the elasticity tensor via [pymatgen](https://github.com/materialsproject/pymatgen). As an example we will do this for three samples: Aluminium, Diamond, and a Carbon-nanotube.

Using the ASE we can build each sample using the utility functions in ```ase.build```.

Firstly Aluminium can be generated using ```ase.build.bulk```.

In [ ]:
al = bulk("Al", crystalstructure="fcc")*(3,3,3)

write("../data/Aluminium.xyz", al)

v=WeasWidget()
v.from_ase(al)
v

Next we can build Diamond,

In [ ]:
diamond = Diamond('C')*(2,2,2)

write("../data/Diamond.xyz", diamond)

v=WeasWidget()
v.from_ase(diamond)
v

And finally the Carbon-nanotube,

In [ ]:
nt = nanotube(6, 6, length=4)
# Place into a box
nt.cell = [nt.cell[2][2], nt.cell[2][2], nt.cell[2][2]]
nt.pbc = [True, True, True]
write("../data/Carbon-nanotube.xyz", nt)
v=WeasWidget()
v.from_ase(nt)
v

### Elasticity calculations

```janus-core``` includes the elasticity CLI command for calculating the elasticity tensor. This is done by first creating a set of deformed (strained) structures. The stress tensor tensor ($\sigma$) is then calculated for each of these deformed structures. Finally the elasticity tensor $C^{ijkl}$ is estimated from the relationship between stress and strain. Or mathematically:

$$ \sigma^{ij} = C^{ijkl} E^{kl} $$

To find $C^{ijkl}$ we can use the ```janus elasticity``` CLI command As with other ```janus_core``` commands we can via its help message for more information:

In [ ]:
! janus elasticity --help

To calculate the elasticity tensor we can use the following CLI command for our Aluminium sample.

In [ ]:
! janus elasticity --arch mace_mp --struct ../data/Aluminium.xyz --no-tracker

The results will appear ```janus_results/Aluminium-elastic_tensor.dat```, which includes the components of the $3\times3\times3\times3$ in (row-major) Voigt reduced form ($6\times6$) which is preceded by various derived values such as the shear and bulk moduli.

In [ ]:
! cat janus_results/Aluminium-elastic_tensor.dat

The experimental bulk, shear, and Young's moduli are approximately 76 GPa, 26 GPa, and 68 GPa for Aluminium. Our results are in the right ball park.

For elasticity the main options for user control are the magnitudes of the applied shear and normal strains as well as their number.

By default 4 shear and 4 normal strains are applied. Split equally positively and negatively, and along each possible direction. This means $2\times 4 \times 3 = 24$ total stress calculations.

For example we can increase the shear and normal magnitudes, and apply 16 strains of each type as follows:

In [ ]:
! janus elasticity --arch mace_mp --struct ../data/Aluminium.xyz --no-tracker --n-strains 16 --shear-magnitude 0.3 --normal-magnitude 0.2

With the results now being different.

In [ ]:
! cat janus_results/Aluminium-elastic_tensor.dat

In the same way we can simply calculate the data for our Diamond and Carbon nano-tube samples,

In [ ]:
! janus elasticity --arch mace_mp --struct ../data/Diamond.xyz --no-tracker --write-structures

In [ ]:
! cat janus_results/Diamond-elastic_tensor.dat

Noticed that we supplied also the option ```--write-structures```. This generates two new files in ```janus_results```: ```Diamond-generated.extxyz``` and ```Diamond-minimized-structure.extxyz```.

The latter is simply our initial diamond structure, but geometry optimized. The former is the set of strained structures used to calculate the elasticity. We can read the file and observe the impact of the strains like so

In [ ]:
v=WeasWidget()
diamond_strained = read("janus_results/Diamond-generated.extxyz", index=":")
v.from_ase(diamond_strained)
v

In [ ]:
! janus elasticity --arch mace_mp --struct ../data/Carbon-nanotube.xyz --no-tracker

In [ ]:
! cat janus_results/Carbon-nanotube-elastic_tensor.dat

We can examine the results to asses the comparative stiffness of the materials, do the results line up with experiments?

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, layout='constrained')
for material, marker in zip(('Aluminium', 'Diamond', 'Carbon-nanotube'), "o^s"):
    tensor = np.loadtxt(f"janus_results/{material}-elastic_tensor.dat")
    bulk, shear, youngs = tensor[[0, 3, 6]]

    theta = [i*2.0*np.pi/3 for i in range(3)]
    ax.scatter(theta, [bulk, shear,youngs], marker=marker, label=material)
    ax.set_xticks(theta, ("Bulk", "Shear", "Youngs"))
fig.legend(loc='outside upper right')
fig.suptitle("Elastic modulii for our samples")